In [ ]:
import pandas as pd
from bs4 import BeautifulSoup
import requests
import time
import re

%matplotlib inline

from user_agent import generate_user_agent
headers = {'User-Agent': generate_user_agent(device_type="desktop", os=('mac', 'linux'))}

pd.set_option('max_colwidth', -1)

In [ ]:
url = 'https://www.azlyrics.com/c/cohen.html'

raw_html = requests.get(url, headers=headers)
soup_doc = BeautifulSoup(raw_html.content, 'html.parser')

In [ ]:
song_urls = []
for song_link in soup_doc.findAll('a')[31:-7]:
    song_dict = {}
    if song_link.text != '':
        song_dict['title'] = song_link.text
        song_dict['url'] = 'https://www.azlyrics.com/lyrics/'+song_link.attrs['href']
    song_urls.append(song_dict)

In [ ]:
df_songs = pd.DataFrame(song_urls)
df_songs.head()

In [ ]:
df_songs.dropna(inplace=True)
urls = df_songs.url

In [ ]:
songs = []
for url in urls:
    song_dict = {}
    raw_html = requests.get(url, headers=headers)
    soup_doc = BeautifulSoup(raw_html.content, 'html.parser')
    song_dict['title'] = soup_doc.findAll('b')[1].text
    song_dict['album'] = soup_doc.findAll('b')[2].text
    song_dict['year'] = soup_doc.findAll('b')[2].next.next
    song_dict['lyrics'] = soup_doc.findAll('div')[21]
    songs.append(song_dict)
    time.sleep(1)

In [ ]:
songs[-1]

In [ ]:
urls[78:]

In [ ]:
for url in urls[78:-3]: #Drop the last three, not released on an album.
    song_dict = {}
    raw_html = requests.get(url, headers=headers, timeout=5)
    soup_doc = BeautifulSoup(raw_html.content, 'html.parser')
    song_dict['title'] = soup_doc.findAll('b')[1].text
    song_dict['album'] = soup_doc.findAll('b')[2].text
    song_dict['year'] = soup_doc.findAll('b')[2].next.next
    song_dict['lyrics'] = soup_doc.findAll('div')[21]
    songs.append(song_dict)
    time.sleep(1)

In [ ]:
df = pd.DataFrame(songs)

In [ ]:
df.lyrics = df.lyrics.astype(str)
df.lyrics = df.lyrics.str.replace('<div>\n<!-- Usage of azlyrics.com content by any third-party lyrics provider is prohibited by our licensing agreement. Sorry about that. -->\r\n','')
df.lyrics = df.lyrics.str.replace('<br/>','')
df.lyrics = df.lyrics.str.replace('</div>','')
df.album = df.album.str.replace('"','')
df.title = df.title.str.replace('"','')
df.year = df.year.str.replace('(','')
df.year = df.year.str.replace(')','')

In [ ]:
df.year.value_counts()

In [ ]:
df.to_csv('avalanche_cleaned.csv', index=False)